# 走行

このNotebookで自動走行をおこないます。

## Jetsonの認識

In [1]:
import Jetson.GPIO as GPIO

BOARD_NAME = GPIO.gpio_pin_data.get_data()[0]

mode_descriptions = {
    "JETSON_NX": ["15W_2CORE", "15W_4CORE", "15W_6CORE", "10W_2CORE", "10W_4CORE"],
    "JETSON_XAVIER": ["MAXN", "MODE_10W", "MODE_15W", "MODE_30W"],
    "JETSON_NANO": ["MAXN", "5W"],
    "JETSON_ORIN": ["MAXN", "MODE_15W", "MODE_30W", "MODE_40W"],
    "JETSON_ORIN_NANO": ["MODE_15W", "MODE_7W"]
}

product_names = {
    "JETSON_NX": "Jetson Xavier NX",
    "JETSON_XAVIER": "Jetson AGX Xavier",
    "JETSON_NANO": "Jetson Nano",
    "JETSON_ORIN": "Jetson AGX Orin",
    "JETSON_ORIN_NANO": "Jetson Orin Nano"
}

# ボードごとのI2Cバス番号と初期Powerモードを定義する
board_settings = {
    "JETSON_NX": (8, 3),
    "JETSON_XAVIER": (8, 2),
    "JETSON_NANO": (1, 0),
    "JETSON_ORIN": (7, 0),
    "JETSON_ORIN_NANO": (7, 0)
}

i2c_busnum, power_mode = board_settings.get(BOARD_NAME, (None, None))
mode_description = mode_descriptions.get(BOARD_NAME, [])
product_name = product_names.get(BOARD_NAME, "未知のボード")

if power_mode is not None and power_mode < len(mode_description):
    mode_str = mode_description[power_mode]
    print("------------------------------------------------------------")
    print(f"{product_name}を認識: I2Cバス番号: {i2c_busnum}, Powerモード: {mode_str}({power_mode})に設定します。")
    print("------------------------------------------------------------")
else:
    print("未知のボードまたは不正なモードです。")

------------------------------------------------------------
Jetson Orin Nanoを認識: I2Cバス番号: 7, Powerモード: MODE_15W(0)に設定します。
------------------------------------------------------------


In [2]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のため電力モードは変更できません。")
else:
    !echo "jetson" | sudo -S nvpmodel -m $power_mode

Docker起動のため電力モードは変更できません。


In [3]:
!echo "jetson" | sudo -S nvpmodel -q

NVPM WARN: power mode is not set!


In [4]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のためjetson_clocksは起動できません。")
else:
    !echo "jetson" | sudo -S jetson_clocks

Docker起動のためjetson_clocksは起動できません。


In [5]:
FPS_60 = 1
FPS_30 = 2
fps_type = 0

if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    fps_type = FPS_60
else:
    fps_type = FPS_30

## ログの表示用 Widget

In [6]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
from IPython.display import clear_output

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value
    
    # UIのクリアと更新
    clear_output(wait=True)

## PWMの値の読み込み

In [7]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

SMBUS='smbus'
BUSNUM=i2c_busnum
SERVO_HZ=60
INITIAL_VALUE=300
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

STEERING_CH = 0
THROTTLE_CH = 1
direction = 0
REVERSE = 0
NORMAL = 1

pwm_front = 0
pwm_back = 0

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    pwm_stop = json_str["pwm_speed"]["stop"]
    pwm_front = json_str["pwm_speed"]["front"]
    pwm_back = json_str["pwm_speed"]["back"]
    pwm_left = json_str["pwm_steering"]["left"]
    pwm_center = json_str["pwm_steering"]["center"]
    pwm_right = json_str["pwm_steering"]["right"]

    
if pwm_front >= pwm_back:
    direction = REVERSE
else:
    direction = NORMAL
    
PCA9685.set_channel_value(STEERING_CH, pwm_center)
PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)

## カメラの読込

この部分でエラーが発生する場合は、Jetsonの再起動をお願いします。<br>
それでも、カメラが認識できない場合は、ケーブルの接続確認をしてください。


In [8]:
from jetcam.csi_camera import CSICamera
camera = None

if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    camera = CSICamera(width=224, height=224, capture_fps=60)
else:
    camera = CSICamera(width=224, height=224, capture_fps=30)

## 走行処理

In [9]:
import torch
import threading
from utils import preprocess
import subprocess
import cv2
import time
from torch2trt import TRTModule
import subprocess
import datetime
import torch.nn.functional as F

IMG_WIDTH=224

model_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_time_widget = ipywidgets.Label(description='作成日時')
load_button = ipywidgets.Button(description='走行モデルを読込み')
model_c_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_c_time_widget = ipywidgets.Label(description='作成日時')
load_c_button = ipywidgets.Button(description='環境モデルを読込み')
run_button = ipywidgets.Button(description='走行開始')
stop_button = ipywidgets.Button(description='停止')
pwm_left_widget = ipywidgets.IntText(value=pwm_left,description='PWM 左')
pwm_center_widget = ipywidgets.IntText(value=pwm_center,description='PWM 中央')
pwm_right_widget = ipywidgets.IntText(value=pwm_right,description='PWM 右')
check_left_button = ipywidgets.Button(description='チェック左')
check_center_button = ipywidgets.Button(description='チェック中央')
check_right_button = ipywidgets.Button(description='チェック右')
pwm_stop_widget = ipywidgets.IntText(value=pwm_stop,description='PWM 停止')
name_widget = ipywidgets.Text(description='映像の保存名')
record_box = ipywidgets.Checkbox(False, description='録画')
speed_gain_slider = ipywidgets.FloatSlider(description='Speed gain', min=0.1, max=2.0, step=0.1, value=1.0, orientation='horizontal')
speed_raw_slider = ipywidgets.IntSlider(description='Speed raw', min=1, max=224, step=1, value=80, orientation='horizontal')
steering_gain_slider = ipywidgets.FloatSlider(description='Steering gain', min=0.1, max=2.0, step=0.1, value=1.0, orientation='horizontal')
speed_dropbox = ipywidgets.Dropdown(options=["固定値","推論値"], description='Speed')


record = False
running = False
load = False
speed_ai_flag = False

def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

def handle(x):
    global pwm_right,pwm_left,STEERING_CH,PCA9685
    x = map_rc(x, 224, 0, pwm_right, pwm_left)
    PCA9685.set_channel_value(STEERING_CH, x)
    
def throttle(speed):
    global pwm_front,pwm_back,THROTTLE_CH,PCA9685
    speed = map_rc(speed, 224, 0, pwm_front, pwm_stop)
    PCA9685.set_channel_value(THROTTLE_CH, speed)
    
def live():
    global speed_ai_flag,running,camera,IMG_WIDTH,record,model_trt,model_c_trt,preprocess,pwm_stop,save_dir,num,count,fps_type,FPS_30
    
    count = 1
    num = 1
    frame_count = 0

    start_time = time.time()
    last_lap_time = 0
    lap = 0
    
    while running:
        try:
            image = camera.read()
            if record == True:
                remarked_img = image.copy()
            image = preprocess(image).half()
            output = model_trt(image).detach().cpu().numpy().flatten()
            x = float(output[0]) * steering_gain_slider.value
            y = float(output[1])
            x = int(IMG_WIDTH * (x / 2.0 + 0.5))
            y = int(IMG_WIDTH * (y / 2.0 + 0.5))
            handle(x)

            speed = float(output[3])
            speed = int(IMG_WIDTH * (speed / 2.0 + 0.5)) * speed_gain_slider.value

            if speed_ai_flag == False:
                speed = speed_raw_slider.value
            else:
                if speed > 224:
                    speed = 224        
            throttle(speed)

            output = model_c_trt(image).detach()
            output = F.softmax(output, dim=1).cpu().numpy().flatten()
            category_index = output.argmax()

            # 現在の時間を取得
            current_time = time.time()

            # Lapの判定と5秒間のスキップロジック
            if current_time - last_lap_time > 5:  # 5秒経過後に判定を再開
                if category_index == 0:
                    if last_detect == 0:
                        detect_count += 1
                        if detect_count == 30:
                            lap += 1
                            detect_count = 0
                            last_lap_time = current_time  # Lapの時間を更新
                    else:
                        detect_count = 1
                else:
                    detect_count = 0

            last_detect = category_index

            # Lap 4週目カウントで走行を停止する
            if lap >= 4:
                running = False

            if record == True:
                if fps_type == FPS_30:
                    name = "0_0_{:0=5}.jpg".format(count)
                    image_path = os.path.join(save_dir, name)
                    cv2.imwrite(image_path, remarked_img)
                else:
                    if count % 2 == 0:
                        name = "0_0_{:0=5}.jpg".format(num)
                        image_path = os.path.join(save_dir, name)
                        cv2.imwrite(image_path, remarked_img)
                        num += 1
            count += 1
            frame_count += 1


            if time.time() - start_time > 3.0:
                fps = frame_count / 3.0
                speed_type = ""
                if speed_ai_flag == False:
                    speed_type = f"(固定)"
                else:
                    speed_type = f"(推論)"
                write_log(f"Lap: {lap}, Speed: {speed} {speed_type}, Steering Gain: {steering_gain_slider.value}, FPS: {fps:.1f} (3秒ごとに表示)")
                frame_count = 0
                start_time = time.time()        
        except Exception as e:
            write_log(f"Error:{e}")
            
    if record == True:
        if fps_type == FPS_30:
            write_log(f"{fps_type}:画像を{count}枚保存しました。")
        else:
            write_log(f"{fps_type}:画像を{num}枚保存しました。")
        
def run(change):
    global running,execute_thread,name_widget,save_dir,start_time,load,load_c
    
    if load == False or load_c == False:
        write_log("モデルが読み込まれていません")
        return

    if running == False:
        if record == True:
            if name_widget.value != "":
                save_dir = "camera/" + name_widget.value + "/xy/" 
                if not os.path.exists(save_dir):
                    subprocess.call(['mkdir', '-p', save_dir])
                write_log(save_dir + "にデータを保存します。")            

                running = True
                execute_thread = threading.Thread(target=live)
                execute_thread.start()
                start_time = time.time()
                write_log("AIが起動しました。")
            else:
                write_log("【Error】 映像の保存先を入力してください。")
        else:
            running = True
            execute_thread = threading.Thread(target=live)
            execute_thread.start()
            start_time = time.time()
            write_log("AIが起動しました。") 

def stop(change):
    global running,execute_thread,end_time,start_time,count,pwm_stop
    if running == True:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        try:
            end_time = time.time() - start_time
            fps = count/int(end_time)
            process_time = int((end_time/count)*1000)
        except:
            fps = -1
            process_time = -1
        
        write_log("AIを停止しました。")
        write_log("処理結果:FPS: " + str(round(fps,2)) + ",処理回数: " + str(count) + ",　処理時間(1回平均値): " + str(process_time) + " ms")
        running = False
        execute_thread.join()
    else:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        write_log("現在AIは動いていません。")

def load_model(change):
    global model_trt,load
    try:
        write_log(model_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_trt = TRTModule()
        model_trt.load_state_dict(torch.load(model_widget.value))
        write_log(model_widget.value + "の読込に成功しました。")
        load = True
    except Exception as e:
        write_log(f"【Error】 {e} : {model_widget.value} の読込に失敗しました。")

load_button.on_click(load_model)      

def load_c_model(change):
    global model_c_trt,load_c
    try:
        write_log(model_c_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_c_trt = TRTModule()
        model_c_trt.load_state_dict(torch.load(model_c_widget.value))
        write_log(model_c_widget.value + "の読込に成功しました。")
        load_c = True
    except Exception as e:
        write_log(f"【Error】 {e} : {model_c_widget.value} の読込に失敗しました。")

load_c_button.on_click(load_c_model) 

def model_list(type):
    try:
        files = glob.glob(type + '/*.pth', recursive=True)
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        if type == "model_trt":
            model_widget.options = files
            model_time_widget.value = s
        elif type == "model_c_trt":
            model_c_widget.options = files
            model_c_time_widget.value = s
    except Exception as e:
        model_widget.options = []
        model_c_widget.options = []
        write_log(f"Error:{e}")
model_list("model_trt")
model_list("model_c_trt")

def change_file(change):
    file = model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_time_widget.value = s
model_widget.observe(change_file)

def change_c_file(change):
    file = model_c_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_c_time_widget.value = s
model_c_widget.observe(change_c_file)


def on_video(change):
    global record
    record^=True

SpeedInference = False

def on_fixed_value_change(change):
    global speed_ai_flag, speed_gain_slider, speed_raw_slider
    if change['new'] == "固定値":
        speed_gain_slider.layout.visibility = 'hidden'
        speed_raw_slider.layout.visibility = 'visible'
        speed_gain_slider.disabled = True
        speed_raw_slider.disabled = False
        speed_ai_flag = False
    elif change['new'] == "推論値":
        speed_gain_slider.layout.visibility = 'visible'
        speed_raw_slider.layout.visibility = 'hidden'
        speed_gain_slider.disabled = False
        speed_raw_slider.disabled = True
        speed_ai_flag = True

speed_gain_slider.layout.visibility = 'hidden'
speed_raw_slider.layout.visibility = 'visible'
speed_gain_slider.disabled = True
speed_raw_slider.disabled = False
speed_dropbox.observe(on_fixed_value_change, names='value')

run_button.on_click(run)
stop_button.on_click(stop)
record_box.observe(on_video)

In [10]:
import subprocess
import re

used_memory_widget = ipywidgets.IntText(description='Useメモリ', value=1)
total_memory_widget = ipywidgets.IntText(description='全メモリ', value=1)
memory_button = ipywidgets.Button(description='使用メモリ量の取得')

def get_jetson_nano_memory_usage(event=None):
    command = 'tegrastats'
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        
        mem_usage_pattern = re.compile(r'RAM (\d+)/(\d+)MB')
        
        max_lines_to_read = 10
        for _ in range(max_lines_to_read):
            line = process.stdout.readline()
            if not line:
                break 
            matches = mem_usage_pattern.search(line)
            if matches:
                used_memory_widget.value = int(matches.group(1))
                total_memory_widget.value = int(matches.group(2))
                process.kill()
                return
        
        process.kill()  
        return

    except subprocess.CalledProcessError as e:
        return

get_jetson_nano_memory_usage()
memory_button.on_click(get_jetson_nano_memory_usage)

In [11]:
import time

release_button = ipywidgets.Button(description='Camera開放')


def stop_camera(c):
    global camera
    camera.running = False
    time.sleep(1)
    camera.cap.release()
    write_log("カメラを開放しました。")

release_button.on_click(stop_camera)

走行までの流れは以下の通りです。

1. <b>走行モデルを指定してLoadする</b><br>
2. <b>環境モデルを指定してLoadする</b><br>
3. <b>Speedに固定値, Speedに推論値のいずれかを選択する</b><br>
固定値を選んだ場合は固定の速度で走り続けます。推論値を選んだ推論結果を速度に反映します。速度用のアノテーションは13_annotation.ipynbで追加可能です。<br>
4. <b>[オプション] 走行動画を録画する場合は、録画にチェックマークをいれて、保存ファイル名を指定する</b><br>
./runフォルダに保存<br>
5. <b>走行開始ボタンを押して、プロポの裏側のボタンを押して AIモードで自動走行開始する</b><br>
6. <b>終了時は、停止ボタンを押す</b><br>
録画のチェックマークがついている場合は、停止で録画も終了<br>
Speed Inferenceのチェックマークがついている場合は、スロットル量の推論も有効になる<br>
<br>
カメラが60fpsで動いている場合は16ms以内、カメラが30fpsで動いている場合は、33ms以内での処理完了が正常な挙動となります。

In [12]:
separator = ipywidgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = ipywidgets.HTML('<b>【1.使用する走行モデル】</b> TensoRTに変換済みのモデルをLoadします。')
title2 = ipywidgets.HTML('<b>【2.使用する環境モデル】</b> TensoRTに変換済みのモデルをLoadします。')
title3 = ipywidgets.HTML('<b>【3.Steeringゲイン】</b> Steeringのゲイン調整します。周りが悪い時は値を1.0以上にします。')
title4 = ipywidgets.HTML('<b>【4.速度】</b> 速度は固定値か、推論から反映かが選べます。')
title5 = ipywidgets.HTML('<b>【5.録画】</b> 走行中の映像を録画したい場合はチェックマークを選択し、保存データセット名を指定してください。')
title6 = ipywidgets.HTML('<b>【6.走行の開始】</b> 走行開始を押す前にタイヤが空転していのを確認してください。プロポの裏のボタンを変換しAIモードにすると動き始めます。')
title7 = ipywidgets.HTML('<b>【7.カメラのリリース】</b> 走行を終了する場合は、カメラのリリースを実行し、Notebookをshutdownしてください。')

data_collection_widget = ipywidgets.VBox([
    separator,
    title1,
    ipywidgets.HBox([model_widget,model_time_widget,load_button]),
    process_widget,
    separator,
    title2,
    ipywidgets.HBox([model_c_widget,model_c_time_widget,load_c_button]),
    process_widget,
    separator,
    title3,
    ipywidgets.HBox([steering_gain_slider]),
    separator,
    title4,
    ipywidgets.HBox([speed_dropbox]),
    ipywidgets.HBox([speed_raw_slider,speed_gain_slider]),
    separator,
    title5,
    ipywidgets.HBox([record_box,name_widget]),
    separator,
    title6,
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    ipywidgets.HBox([run_button, stop_button]),
    process_widget,
    separator,
    title7,
    release_button,
    process_widget,
])
display(data_collection_widget)